# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.234112221115                   -0.50    7.0         
  2   -7.250114946850       -1.80       -1.40    1.0   5.23ms
  3   -7.251244188285       -2.95       -2.16    2.0   6.05ms
  4   -7.251313953368       -4.16       -2.39    2.0   6.36ms
  5   -7.251330433203       -4.78       -2.69    1.0   5.26ms
  6   -7.251337843181       -5.13       -3.15    1.0   41.0ms
  7   -7.251338712659       -6.06       -3.72    1.0   5.28ms
  8   -7.251338786734       -7.13       -4.04    3.0   6.96ms
  9   -7.251338797682       -7.96       -4.72    1.0   5.35ms
 10   -7.251338798647       -9.02       -5.06    3.0   7.41ms
 11   -7.251338798696      -10.31       -5.63    1.0   5.58ms
 12   -7.251338798703      -11.16       -5.95    3.0   7.63ms
 13   -7.251338798704      -11.86       -6.45    1.0   5.72ms
 14   -7.251338798704      -12.97       -6.62    3.0   7.40ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.07294796116224415
[ Info: Arnoldi iteration step 2: normres = 0.319517505540526
[ Info: Arnoldi iteration step 3: normres = 1.033714932158315
[ Info: Arnoldi iteration step 4: normres = 0.2939858478885413
[ Info: Arnoldi iteration step 5: normres = 0.5988202821122616
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (6.62e-02, 5.76e-02, 5.52e-01, 2.16e-01, 4.61e-03)
[ Info: Arnoldi iteration step 6: normres = 0.22504187332431128
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.29e-02, 1.64e-01, 8.82e-02, 8.99e-02, 4.45e-02)
[ Info: Arnoldi iteration step 7: normres = 0.1098962723713693
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (6.96e-04, 1.61e-02, 6.93e-03, 8.07e-02, 6.38e-02)
[ Info: Arnoldi iteration step 8: normres = 0.109173911894034
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (3.33e-05, 1